# Fine-tuning RoBERTa model for NLP-Challenge (28-Class problem)




In [1]:
pip install transformers

     |████████████████████████████████| 1.3MB 7.3MB/s 
     |████████████████████████████████| 2.9MB 24.7MB/s 
     |████████████████████████████████| 890kB 55.6MB/s 
     |████████████████████████████████| 1.1MB 55.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=72e03ef8adc9604d54e539faf7fb5b71bea51b8fe666b55ff037f3afadfd067e
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')
%ls "/content/gdrive/My Drive"

Mounted at /content/gdrive
'Colab Notebooks'/   template_submissions.csv   train.json
 DistilBERT.csv      test_distilber_fine.csv    train_label.csv
 metrics/            test.json
 saved_models/       test_roberta_fine.csv


In [4]:
import sys
sys.path.append("/content/gdrive/My Drive/")
from transformers import RobertaTokenizer
from transformers import TFRobertaForSequenceClassification, TFRobertaModel
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

In [5]:
%tensorflow_version 2.x
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

Tensorflow version 2.3.0


In [ ]:
try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.TPUStrategy(tpu)

INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


Running on TPU  ['10.69.211.114:8470']
INFO:tensorflow:Initializing the TPU system: grpc://10.69.211.114:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.69.211.114:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


# The data

In [6]:
PATH = '/content/gdrive/My Drive/'
df = pd.read_json(PATH+'train.json').set_index('Id').loc[:, 'description']
labels = pd.read_csv(PATH+'train_label.csv', index_col=0).loc[:, 'Category'].astype('category').cat.codes

In [ ]:
train_texts = df.tolist()
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
train_encodings = tokenizer(train_texts, padding=True)

Token indices sequence length is longer than the specified maximum sequence length for this model (540 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
train_labels = labels.tolist()
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels))

In [ ]:
def create_model():
  model = TFRobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=28)
  model.layers[0].trainable = False
  model.compile(optimizer="adam", 
              loss=model.compute_loss, 
              metrics=['accuracy'])
  return model

In [ ]:
with tpu_strategy.scope(): # creating the model in the TPUStrategy scope means we will train the model on the TPU
  model = create_model()
model.summary()

Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaForSequenceClassification: ['lm_head']
- This IS expected if you are initializing TFRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "tf_roberta_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
roberta (TFRobertaMainLayer) multiple                  124645632 
_________________________________________________________________
classifier (TFRobertaClassif multiple                  612124    
Total params: 125,257,756
Trainable params: 612,124
Non-trainable params: 124,645,632
_________________________________________________________________


In [ ]:
model.fit(train_dataset.batch(32),
          epochs=12)

Epoch 1/12
Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.


Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.


   2/6788 [..............................] - ETA: 18:25:53 - loss: 3.2021 - accuracy: 0.1875    WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0053s vs `on_train_batch_end` time: 0.0816s). Check your callbacks.


6788/6788 [==============================] - 628s 92ms/step - loss: 1.1003 - accuracy: 0.6820
Epoch 2/12
6788/6788 [==============================] - 605s 89ms/step - loss: 0.9685 - accuracy: 0.7148
Epoch 3/12
6788/6788 [==============================] - 602s 89ms/step - loss: 0.9446 - accuracy: 0.7193
Epoch 4/12
6788/6788 [==============================] - 603s 89ms/step - loss: 0.9315 - accuracy: 0.7222
Epoch 5/12
6788/6788 [==============================] - 604s 89ms/step - loss: 0.9174 - accuracy: 0.7262
Epoch 6/12
6788/6788 [==============================] - 602s 89ms/step - loss: 0.9111 - accuracy: 0.7267
Epoch 7/12
6788/6788 [==============================] - 603s 89ms/step - loss: 0.9078 - accuracy: 0.7286
Epoch 8/12
6788/6788 [==============================] - 603s 89ms/step - loss: 0.8977 - accuracy: 0.7309
Epoch 9/12
6788/6788 [==============================] - 603s 89ms/step - loss: 0.8948 - accuracy: 0.7305
Epoch 10/12
6788/6788 [==============================] - 604s 89ms

In [ ]:
with tpu_strategy.scope(): # creating the model in the TPUStrategy scope means we will train the model on the TPU
    optimizer = tf.keras.optimizers.Adam(learning_rate=1.5e-5)
    model.trainable=True
    model.compile(optimizer=optimizer, 
              loss=model.compute_loss, 
              metrics=['accuracy'])
model.summary()

Model: "tf_roberta_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
roberta (TFRobertaMainLayer) multiple                  124645632 
_________________________________________________________________
classifier (TFRobertaClassif multiple                  612124    
Total params: 125,257,756
Trainable params: 125,257,756
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(train_dataset.batch(48),
          epochs=2)

Epoch 1/2


   2/4525 [..............................] - ETA: 25:14:56 - loss: 1.0576 - accuracy: 0.6458WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0102s vs `on_train_batch_end` time: 0.3257s). Check your callbacks.


4525/4525 [==============================] - 1566s 346ms/step - loss: 0.5906 - accuracy: 0.8220
Epoch 2/2
4525/4525 [==============================] - 1530s 338ms/step - loss: 0.4735 - accuracy: 0.8541


In [ ]:
save_directory = PATH+"saved_models/jet4_RoBERTa"
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

('/content/gdrive/My Drive/saved_models/jet4_RoBERTa/tokenizer_config.json',
 '/content/gdrive/My Drive/saved_models/jet4_RoBERTa/special_tokens_map.json',
 '/content/gdrive/My Drive/saved_models/jet4_RoBERTa/vocab.json',
 '/content/gdrive/My Drive/saved_models/jet4_RoBERTa/merges.txt',
 '/content/gdrive/My Drive/saved_models/jet4_RoBERTa/added_tokens.json')

In [7]:
PATH = '/content/gdrive/My Drive/'
save_directory = PATH+"saved_models/jet4_RoBERTa"
loaded_tokenizer = RobertaTokenizer.from_pretrained(save_directory)
loaded_model = TFRobertaForSequenceClassification.from_pretrained(save_directory)

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at /content/gdrive/My Drive/saved_models/jet4_RoBERTa.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


In [8]:
test_texts = pd.read_json(PATH+'test.json').set_index('Id').loc[:, 'description'].tolist()
len(test_texts)

54300

In [9]:
y = []
for i, text in enumerate(test_texts):
    predict_input = loaded_tokenizer.encode(text,truncation=True,padding=True,return_tensors="tf")
    output = loaded_model(predict_input)[0]
    y.append(tf.argmax(output, axis=1).numpy()[0])
    print(i,"/ 54300",round(i*100/54300,2))
y_pred = pd.Series(y, name='y_pred')

Streaming output truncated to the last 5000 lines.
49300 / 54300 90.79
49301 / 54300 90.79
49302 / 54300 90.8
49303 / 54300 90.8
49304 / 54300 90.8
49305 / 54300 90.8
49306 / 54300 90.8
49307 / 54300 90.8
49308 / 54300 90.81
49309 / 54300 90.81
49310 / 54300 90.81
49311 / 54300 90.81
49312 / 54300 90.81
49313 / 54300 90.82
49314 / 54300 90.82
49315 / 54300 90.82
49316 / 54300 90.82
49317 / 54300 90.82
49318 / 54300 90.83
49319 / 54300 90.83
49320 / 54300 90.83
49321 / 54300 90.83
49322 / 54300 90.83
49323 / 54300 90.83
49324 / 54300 90.84
49325 / 54300 90.84
49326 / 54300 90.84
49327 / 54300 90.84
49328 / 54300 90.84
49329 / 54300 90.85
49330 / 54300 90.85
49331 / 54300 90.85
49332 / 54300 90.85
49333 / 54300 90.85
49334 / 54300 90.85
49335 / 54300 90.86
49336 / 54300 90.86
49337 / 54300 90.86
49338 / 54300 90.86
49339 / 54300 90.86
49340 / 54300 90.87
49341 / 54300 90.87
49342 / 54300 90.87
49343 / 54300 90.87
49344 / 54300 90.87
49345 / 54300 90.87
49346 / 54300 90.88
49347 / 54300 9

In [13]:
submission = pd.read_csv(PATH+'template_submissions.csv')
submission.Category = y_pred
submission

,Id,Category
0,0,6
1,1,20
2,2,22
3,3,19
4,4,19
...,...,...
54295,54295,19
54296,54296,19
54297,54297,19
54298,54298,26


In [14]:
submission.to_csv(PATH+'RoBERTa.csv', index=False)